In [1]:
import pandas as pd
import numpy as np
from datascience import *
import csv
from functools import reduce
import math
import random

In [2]:
filepath = r"/1976-2022-house.csv";
df = pd.read_csv(filepath).fillna('INDEPENDENT')
house_tbl = Table.from_df(df)
house_reduced = house_tbl.where('year', are.above(2003)).where('totalvotes', are.above(1)).where('stage', 'GEN').where('writein', False).select('year', 'state', 'district', 'stage', 'candidate', 'party','candidatevotes', 'totalvotes')
house_reduced = house_reduced.with_column('pct', house_reduced.column('candidatevotes') / house_reduced.column('totalvotes'))
all_states = reduce(lambda re, x: re+[x] if x not in re else re, house_reduced.where('year', 2022).column('state').tolist(), [])
states2020 = ['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'COLORADO', 'DELAWARE', 'GEORGIA', 'IDAHO', 'ILLINOIS', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MONTANA', 'NEBRASKA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 'NORTH CAROLINA', 'OKLAHOMA', 'OREGON', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'VIRGINIA', 'WEST VIRGINIA', 'WYOMING']

In [3]:
filepath = r"/senate_polls_historical.csv";
df = pd.read_csv(filepath).fillna(0)
polls_past = Table.from_df(df)

In [4]:
filepath = r"/1976-2020-senate.csv";
df = pd.read_csv(filepath)
senate_results = Table.from_df(df)

In [5]:
# Train empirics based model - data from 2020

def calc_ewma(lst, alpha = 0.73):
    if len(lst) == 1:
      return lst[0]
    return alpha * lst[0] + (1 - alpha) * calc_ewma(lst[1:], alpha)

dem_vote_share = [0.466,0.52,0.529,0.448,0.484,0.449,0.473,0.529]
gop_vote_share = [0.492,0.441,0.424,0.514,0.471,0.507,0.483,0.443]
ind_vote_share = [0.042,0.04,0.047,0.038,0.045,0.045,0.043,0.028]
dem_generic20 = 0.49869617
gop_generic20 = 0.42613349
est_ind_vote20 = calc_ewma(ind_vote_share[::-1], alpha = 0.73)
est_dem_vote20 = (dem_generic20 / (dem_generic20 + gop_generic20)) * (1 - est_ind_vote20)
est_gop_vote20 = (gop_generic20 / (dem_generic20 + gop_generic20)) * (1 - est_ind_vote20)

def get_party_avg(state):
  house_state = house_reduced.where('state', state)
  parties = reduce(lambda re, x: re+[x] if x not in re else re, house_state.column('party').tolist(), [])[:20]
  one_avg = []
  all_avgs = []
  states = []
  for party in parties:
    for year in range(2004, 2020, 2):
      house_yr = house_state.where('year', year)
      house_yr_grouped = house_yr.group('party', sum)
      total_votes = max(house_yr_grouped.column('totalvotes sum'))
      party_tbl = house_yr_grouped.where('party', party).column('candidatevotes sum').tolist()
      if party_tbl == []:
        party_votes = 0
      else:
        party_votes = party_tbl[0] / total_votes
      if party == 'DEMOCRAT':
        vote_diff = party_votes - dem_vote_share[int((year - 2004) / 2)]
      elif party == 'REPUBLICAN':
        vote_diff = party_votes - gop_vote_share[int((year - 2004) / 2)]
      else:
        vote_diff = party_votes - ind_vote_share[int((year - 2004) / 2)]
      one_avg.append(vote_diff)
    party_ewma = calc_ewma(one_avg[::-1])
    if party == 'DEMOCRAT':
      party_proj = party_ewma + est_dem_vote20
    elif party == 'REPUBLICAN':
      party_proj = party_ewma + est_gop_vote20
    else:
      party_proj = party_ewma + est_ind_vote20
    all_avgs.append(party_proj)
    states.append(state)
  return Table().with_columns('State',states,'Party', parties, 'Pct', all_avgs)

def get_empiric():
  total_tbl = Table().with_columns('State', [], 'Party', [], 'Pct', [])
  for i in all_states:
    one_state = get_party_avg(i)
    total_tbl.append(one_state)
  return total_tbl

empirics2020 = get_empiric()

In [6]:
# Train polls based model - data from 2020

def state_polls(state):
  reduced_polls = polls_past.where('state', state.capitalize()).where('cycle', 2020).where('poll_id', are.between(68011, 72936)).select('poll_id', 'pollster_id', 'pollster', 'start_date', 'end_date', 'race_id', 'party', 'candidate_name', 'pct')
  candidates = reduce(lambda re, x: re+[x] if x not in re else re, reduced_polls.column('candidate_name').tolist(), [])
  all_pcts = []
  cand_state = []
  cand_party = []
  for candidate in candidates:
    cand_tbl = reduced_polls.where('candidate_name', candidate)
    cand_pct = cand_tbl.column('pct').tolist()
    cand_ewma = calc_ewma(cand_pct, alpha = 0.76) / 100
    all_pcts.append(cand_ewma)
    cand_state.append(state.upper())
    if cand_tbl.column('party').item(0) == 'DEM':
      cand_party.append('DEMOCRAT')
    elif cand_tbl.column('party').item(0) == 'REP':
      cand_party.append('REPUBLICAN')
    elif cand_tbl.column('party').item(0) == 'CON':
      cand_party.append('CONSTITUTION')
    elif cand_tbl.column('party').item(0) == 'GRE':
      cand_party.append('GREEN')
    elif cand_tbl.column('party').item(0) == 'LIB':
      cand_party.append('LIBERTARIAN')
    else:
      cand_party.append('INDEPENDENT')
  final_tbl = Table().with_columns('State', cand_state, 'Party', cand_party,'Candidate', candidates, 'Pct', all_pcts)
  return final_tbl

def get_polls(state_lst):
  full_tbl = Table().with_columns('State', [],'Party',[], 'Candidate', [], 'Pct', [])
  for i in state_lst:
    one_state = state_polls(i)
    full_tbl = full_tbl.append(one_state)
  return full_tbl

polls2020 = get_polls(states2020)

In [7]:
# Test both models together
empirics_and_polls = polls2020.join(['State', 'Party'], empirics2020, ['State', 'Party'])
senate2020 = senate_results.where('year', 2020).select('candidate', 'candidatevotes', 'totalvotes')
senate2020 = senate2020.with_column('pct', senate2020.column('candidatevotes') / senate2020.column('totalvotes')).select('candidate', 'pct')
candidates = [x.upper() for x in empirics_and_polls.column('Candidate').tolist()]
empirics_and_polls = empirics_and_polls.with_column('Name', candidates).select('Name', 'Pct', 'Pct_2')
proj_and_results = empirics_and_polls.join('Name', senate2020, 'candidate')
polls_lst = proj_and_results.column('Pct')
empirics_lst = proj_and_results.column('Pct_2')
results_lst = proj_and_results.column('pct')

def test_weights(empirics_wt, polls_wt):
  difference = ((empirics_wt * empirics_lst + polls_wt * polls_lst) - results_lst) ** 2
  return np.mean(difference)

weights = []
diffs = []
for i in np.arange(0,1,0.01):
  one_diff = test_weights(i, 1 - i)
  diffs.append(one_diff)
  weights.append(i)

Table().with_columns('Empiric Weight', weights, 'Diff', diffs).show()

Empiric Weight,Diff
0,0.0025193
0.01,0.00252835
0.02,0.00254488
0.03,0.00256886
0.04,0.00260031
0.05,0.00263923
0.06,0.00268561
0.07,0.00273946
0.08,0.00280078
0.09,0.00286956


In [8]:
filepath = r"/senate_polls (2).csv";
df = pd.read_csv(filepath).fillna('INDEPENDENT')
polls_current = Table.from_df(df)
senate_state_list = ['ARIZONA', 'CALIFORNIA', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'HAWAII', 'INDIANA', 'INDIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 'NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH DAKOTA', 'OHIO', 'PENNSYLVANIA', 'RHODE ISLAND', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING']

In [9]:
# Test 2024

dem_vote_share = [0.466,0.52,0.529,0.448,0.484,0.449,0.473,0.529,0.503,0.473]
gop_vote_share = [0.492,0.441,0.424,0.514,0.471,0.507,0.483,0.443,0.472,0.5]
ind_vote_share = [0.042,0.04,0.047,0.038,0.045,0.045,0.043,0.028,0.025,0.027]
dem_generic20 = 0.462
gop_generic20 = 0.451
est_ind_vote20 = calc_ewma(ind_vote_share[::-1], alpha = 0.73)
est_dem_vote20 = (dem_generic20 / (dem_generic20 + gop_generic20)) * (1 - est_ind_vote20)
est_gop_vote20 = (gop_generic20 / (dem_generic20 + gop_generic20)) * (1 - est_ind_vote20)

def get_party_avg(state):
  house_state = house_reduced.where('state', state)
  parties = reduce(lambda re, x: re+[x] if x not in re else re, house_state.column('party').tolist(), [])[:20]
  one_avg = []
  all_avgs = []
  states = []
  for party in parties:
    for year in range(2004, 2024, 2):
      house_yr = house_state.where('year', year)
      house_yr_grouped = house_yr.group('party', sum)
      total_votes = max(house_yr_grouped.column('totalvotes sum'))
      party_tbl = house_yr_grouped.where('party', party).column('candidatevotes sum').tolist()
      if party_tbl == []:
        party_votes = 0
      else:
        party_votes = party_tbl[0] / total_votes
      if party == 'DEMOCRAT':
        vote_diff = party_votes - dem_vote_share[int((year - 2004) / 2)]
      elif party == 'REPUBLICAN':
        vote_diff = party_votes - gop_vote_share[int((year - 2004) / 2)]
      else:
        vote_diff = party_votes - ind_vote_share[int((year - 2004) / 2)]
      one_avg.append(vote_diff)
    party_ewma = calc_ewma(one_avg[::-1])
    if party == 'DEMOCRAT':
      party_proj = party_ewma + est_dem_vote20
    elif party == 'REPUBLICAN':
      party_proj = party_ewma + est_gop_vote20
    else:
      party_proj = party_ewma + est_ind_vote20
    all_avgs.append(party_proj)
    states.append(state)
  return Table().with_columns('State',states,'Party', parties, 'Pct', all_avgs)

def get_empiric():
  total_tbl = Table().with_columns('State', [], 'Party', [], 'Pct', [])
  for i in senate_state_list:
    one_state = get_party_avg(i)
    total_tbl.append(one_state)
  return total_tbl

empirics2024 = get_empiric()

def state_polls(state):
  reduced_polls = polls_current.where('state', state.capitalize()).where('cycle', 2024).where('poll_id', are.above(87697)).select('poll_id', 'pollster_id', 'pollster', 'start_date', 'end_date', 'race_id', 'party', 'candidate_name', 'pct')
  candidates = reduce(lambda re, x: re+[x] if x not in re else re, reduced_polls.column('candidate_name').tolist(), [])
  all_pcts = []
  cand_state = []
  cand_party = []
  for candidate in candidates:
    cand_tbl = reduced_polls.where('candidate_name', candidate)
    cand_pct = cand_tbl.column('pct').tolist()
    cand_ewma = calc_ewma(cand_pct, alpha = 0.76) / 100
    all_pcts.append(cand_ewma)
    cand_state.append(state.upper())
    if cand_tbl.column('party').item(0) == 'DEM':
      cand_party.append('DEMOCRAT')
    elif cand_tbl.column('party').item(0) == 'REP':
      cand_party.append('REPUBLICAN')
    elif cand_tbl.column('party').item(0) == 'CON':
      cand_party.append('CONSTITUTION')
    elif cand_tbl.column('party').item(0) == 'GRE':
      cand_party.append('GREEN')
    elif cand_tbl.column('party').item(0) == 'LIB':
      cand_party.append('LIBERTARIAN')
    else:
      cand_party.append('INDEPENDENT')
  final_tbl = Table().with_columns('State', cand_state, 'Party', cand_party,'Candidate', candidates, 'Pct', all_pcts)
  return final_tbl

def get_polls(state_lst):
  full_tbl = Table().with_columns('State', [],'Party',[], 'Candidate', [], 'Pct', [])
  for i in state_lst:
    one_state = state_polls(i)
    full_tbl = full_tbl.append(one_state)
  return full_tbl

polls2024 = get_polls(senate_state_list)
polls2024.show()

State,Party,Candidate,Pct
ARIZONA,DEMOCRAT,Ruben Gallego,0.496
ARIZONA,REPUBLICAN,Kari Lake,0.386
CALIFORNIA,DEMOCRAT,Adam B. Schiff,0.663
CALIFORNIA,REPUBLICAN,Steve Garvey,0.337
FLORIDA,DEMOCRAT,Debbie Mucarsel-Powell,0.429424
FLORIDA,REPUBLICAN,Rick Scott,0.47288
MICHIGAN,DEMOCRAT,Elissa Slotkin,0.467904
MICHIGAN,REPUBLICAN,Mike Rogers,0.388352
MONTANA,DEMOCRAT,Jon Tester,0.45108
MONTANA,REPUBLICAN,Tim Sheehy,0.50224


In [10]:
empirics_and_polls = polls2024.join(['State', 'Party'], empirics2024, ['State', 'Party'])
polls_lst = empirics_and_polls.column('Pct')
empirics_lst = empirics_and_polls.column('Pct_2')
pre_prob = empirics_and_polls.with_column('Proj Vote', polls_lst * 0.84 + empirics_lst * 0.16).sort('Proj Vote', descending = True).sort('State')

all_diffs = []
for state in reduce(lambda re, x: re+[x] if x not in re else re, pre_prob.column('State').tolist(), []):
  state_tbl = pre_prob.where('State', state)
  proj_vote_state = state_tbl.column('Proj Vote')
  diff = []
  for i in range(state_tbl.num_rows):
    if i == 0:
      diff.append(proj_vote_state.item(0) - proj_vote_state.item(1))
    else:
      diff.append(proj_vote_state.item(i) - proj_vote_state.item(0))
  all_diffs += diff

tbl_w_margins = pre_prob.with_column('Proj Margin', all_diffs)
probabilities = 1 / (1 + 2.718281828459045 ** (-27.5411 * tbl_w_margins.column('Proj Margin')))
total_tbl = tbl_w_margins.with_column('Probability', probabilities)
total_tbl.show()

State,Party,Candidate,Pct,Pct_2,Proj Vote,Proj Margin,Probability
ARIZONA,DEMOCRAT,Ruben Gallego,0.496,0.455122,0.489459,0.0796131,0.899586
ARIZONA,REPUBLICAN,Kari Lake,0.386,0.53504,0.409846,-0.0796131,0.100414
CALIFORNIA,DEMOCRAT,Adam B. Schiff,0.663,0.594696,0.652071,0.318768,0.999846
CALIFORNIA,REPUBLICAN,Steve Garvey,0.337,0.313899,0.333304,-0.318768,0.000153876
FLORIDA,REPUBLICAN,Rick Scott,0.47288,0.556222,0.486215,0.0572028,0.828556
FLORIDA,DEMOCRAT,Debbie Mucarsel-Powell,0.429424,0.426848,0.429012,-0.0572028,0.171444
MICHIGAN,DEMOCRAT,Elissa Slotkin,0.467904,0.509957,0.474633,0.0739637,0.884632
MICHIGAN,REPUBLICAN,Mike Rogers,0.388352,0.465332,0.400669,-0.0739637,0.115368
MONTANA,REPUBLICAN,Tim Sheehy,0.50224,0.52457,0.505813,0.0662567,0.861139
MONTANA,DEMOCRAT,Jon Tester,0.45108,0.379055,0.439556,-0.0662567,0.138861


In [11]:
def one_sim(tbl):
  dem_wins = 0
  gop_wins = 0
  ind_wins = 0
  for state in reduce(lambda re, x: re+[x] if x not in re else re, tbl.column('State').tolist(), []):
    state_tbl = tbl.where('State', state)
    probs = state_tbl.column('Probability').tolist()
    parties = state_tbl.column('Party').tolist()
    rand_num = random.uniform(0, 1)
    total_vote = 0
    wins = []
    for i in range(state_tbl.num_rows):
      if sum(wins) == 1:
        wins.append(0)
      else:
        total_vote += probs[i]
        if rand_num < total_vote:
          wins.append(1)
          if parties[i] == 'DEMOCRAT':
            dem_wins += 1
          elif parties[i] == 'REPUBLICAN':
            gop_wins += 1
          else:
            ind_wins += 1
        else:
          wins.append(0)
  win_tbl = Table().with_columns('Party', ['DEMOCRAT', 'REPUBLICAN', 'OTHER'], 'Wins', [dem_wins, gop_wins, ind_wins])
  return win_tbl
one_sim(total_tbl)

Party,Wins
DEMOCRAT,5
REPUBLICAN,3
OTHER,0
